In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
for i in range(1,101):
    with open('simulationData\sim_'+str(i)+'.csv', 'r' ) as theFile:
        reader = csv.reader(theFile)
        headers = next(reader, None)
        actions = []
        states = []
        rewards = []
        blocks = []
        for line in reader:
            actions.append(int(line[3]))
            states.append([int(line[8])+13*(int(line[0])-1),int(line[9])+13*(int(line[0])-1)])
            rewards.append(int(line[5]))
        block = [
                {
                    'action': np.array([actions])-1,
                    'state': np.array([states]),
                    'reward': np.array([rewards]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]
        if i<91:
            data['S'+str(i)] = block
        else:
            test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
           0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
           0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
           0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
           0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
           0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
           1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
           0, 0, 0, 0, 0, 1]]),
   'state': array([[[  8,   9],
           [ 10,   9],
           [  9,   8],
           [  8,  10],
           [ 10,   8],
           [  9,   8],
           [  8,   9],
           [ 10,   9],
           [ 10,   8],
           [  8,   9],
           [  9,   8],
           [  8,   9],
           [  8,   9],
           [ 10,   9],
           [  9,   8],
           [  9,   8],
           [ 26,  23],
           [ 

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/exploreExploit_blockIndicator/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-12 15:39:57,417 - DeepRL - DEBUG - version control: (None, None)
2021-07-12 15:39:57,419 - DeepRL - DEBUG - learning rate: 0.01
2021-07-12 15:39:57,421 - DeepRL - DEBUG - global iters: 50
2021-07-12 15:39:57,422 - DeepRL - DEBUG - training data-points: 90
2021-07-12 15:39:57,423 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-12 15:40:19,209 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/exploreExploit_blockIndicator/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:40:50,637 - DeepRL - DEBUG - global iter =    0 total obj: 9997.3288
2021-07-12 15:41:07,010 - DeepRL - DEBUG - global iter =    1 total obj: 9988.1875
2021-07-12 15:41:23,070 - DeepRL - DEBUG - global iter =    2 total obj: 9984.6813
2021-07-12 15:41:39,458 - DeepRL - DEBUG - global iter =    3 total obj: 9982.6164
2021-07-12 15:41:55,561 - DeepRL - DEBUG - global iter =    4 total obj: 9961.7922
2021-07-12 15:42:11,731 - DeepRL - DEBUG - global iter =    5 total obj: 9953.7645
2021-07-12 15:42:27,789 - DeepRL - DEBUG - global iter =    6 total obj: 9951.7430
2021-07-12 15:42:44,127 - DeepRL - DEBUG - global iter =    7 total obj: 9948.5490
2021-07-12 15:43:00,291 - DeepRL - DEBUG - global iter =    8 total obj: 9944.8764
2021-07-12 15:43:16,593 - DeepRL - DEBUG - global iter =    9 total obj: 9941.6456
2021-07-12 15:43:32,910 - DeepRL - DEBUG - global iter =   10 total obj: 9933.8705
2021-07-12 15:43:48,799 - DeepRL - DEBUG - global iter =   11 total obj: 9927.0190
2021

INFO:tensorflow:../results/exploreExploit_blockIndicator/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[0, 1, 1, ..., 1, 1, 0],
          [0, 0, 1, ..., 0, 0, 1],
          [0, 1, 0, ..., 0, 1, 1],
          ...,
          [0, 1, 1, ..., 0, 0, 1],
          [1, 0, 0, ..., 0, 1, 0],
          [0, 0, 0, ..., 0, 1, 1]]),
   'action': array([[1, 0, 0, ..., 1, 1, 0],
          [0, 1, 1, ..., 0, 1, 1],
          [0, 0, 0, ..., 1, 0, 1],
          ...,
          [1, 0, 0, ..., 0, 1, 0],
          [0, 0, 1, ..., 0, 0, 0],
          [0, 1, 0, ..., 0, 0, 1]]),
   'state': array([[[  2,   4],
           [  2,   4],
           [  2,   4],
           ...,
           [135, 139],
           [135, 138],
           [139, 138]],
   
          [[  8,   9],
           [ 10,   9],
           [  9,   8],
           ...,
           [141, 132],
           [140, 132],
           [132, 140]],
   
          [[  8,   9],
           [ 10,   9],
           [  9,   8],
           ...,
           [141, 132],
           [140, 132],
           [132, 140]],
   
          ...,
   
          [

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-12 15:52:33,828 - DeepRL - DEBUG - version control: (None, None)
2021-07-12 15:52:33,830 - DeepRL - DEBUG - learning rate: 0.01
2021-07-12 15:52:33,831 - DeepRL - DEBUG - global iters: 50
2021-07-12 15:52:33,833 - DeepRL - DEBUG - training data-points: 90
2021-07-12 15:52:33,835 - DeepRL - DEBUG - test data-points: 10
2021-07-12 15:52:53,227 - DeepRL - DEBUG - opt started...
2021-07-12 15:52:53,229 - DeepRL - DEBUG - started testing...
2021-07-12 15:53:27,978 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blockIndicator/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:53:47,707 - DeepRL - DEBUG - global iter =    0 total obj: 9991.7868
2021-07-12 15:53:57,762 - DeepRL - DEBUG - global iter =    1 total obj: 9983.9028
2021-07-12 15:54:08,372 - DeepRL - DEBUG - global iter =    2 total obj: 9979.7588
2021-07-12 15:54:21,230 - DeepRL - DEBUG - global iter =    3 total obj: 9977.3464
2021-07-12 15:54:34,664 - DeepRL - DEBUG - global iter =    4 total obj: 9971.0784
2021-07-12 15:54:48,897 - DeepRL - DEBUG - global iter =    5 total obj: 9967.3737
2021-07-12 15:55:01,783 - DeepRL - DEBUG - global iter =    6 total obj: 9956.6187
2021-07-12 15:55:14,752 - DeepRL - DEBUG - global iter =    7 total obj: 9947.6470
2021-07-12 15:55:27,656 - DeepRL - DEBUG - global iter =    8 total obj: 9944.7516
2021-07-12 15:55:40,084 - DeepRL - DEBUG - global iter =    9 total obj: 9939.6762
2021-07-12 15:55:40,086 - DeepRL - DEBUG - started testing...
2021-07-12 15:56:07,694 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blockIndicator/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:56:24,146 - DeepRL - DEBUG - global iter =   10 total obj: 9931.1484
2021-07-12 15:56:35,564 - DeepRL - DEBUG - global iter =   11 total obj: 9925.8096
2021-07-12 15:56:45,346 - DeepRL - DEBUG - global iter =   12 total obj: 9916.9534
2021-07-12 15:56:55,369 - DeepRL - DEBUG - global iter =   13 total obj: 9911.0630
2021-07-12 15:57:07,172 - DeepRL - DEBUG - global iter =   14 total obj: 9909.4054
2021-07-12 15:57:20,355 - DeepRL - DEBUG - global iter =   15 total obj: 9904.9089
2021-07-12 15:57:36,400 - DeepRL - DEBUG - global iter =   16 total obj: 9897.4962
2021-07-12 15:57:50,793 - DeepRL - DEBUG - global iter =   17 total obj: 9907.2620
2021-07-12 15:58:03,498 - DeepRL - DEBUG - global iter =   18 total obj: 9902.3866
2021-07-12 15:58:16,345 - DeepRL - DEBUG - global iter =   19 total obj: 9896.0260
2021-07-12 15:58:16,349 - DeepRL - DEBUG - started testing...
2021-07-12 15:58:37,226 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/exploreExploit_blockIndicator/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 16:01:33,293 - DeepRL - DEBUG - global iter =   30 total obj: 9840.7377
2021-07-12 16:01:48,526 - DeepRL - DEBUG - global iter =   31 total obj: 9828.0820
2021-07-12 16:02:02,975 - DeepRL - DEBUG - global iter =   32 total obj: 9828.9529
2021-07-12 16:02:15,410 - DeepRL - DEBUG - global iter =   33 total obj: 9831.3806
2021-07-12 16:02:25,338 - DeepRL - DEBUG - global iter =   34 total obj: 9833.2173
2021-07-12 16:02:35,214 - DeepRL - DEBUG - global iter =   35 total obj: 9809.7462
2021-07-12 16:02:45,051 - DeepRL - DEBUG - global iter =   36 total obj: 9802.9877
2021-07-12 16:02:55,018 - DeepRL - DEBUG - global iter =   37 total obj: 9797.7163
2021-07-12 16:03:05,722 - DeepRL - DEBUG - global iter =   38 total obj: 9819.4569
2021-07-12 16:03:15,874 - DeepRL - DEBUG - global iter =   39 total obj: 9853.6533
2021-07-12 16:03:15,877 - DeepRL - DEBUG - started testing...
2021-07-12 16:03:20,689 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blockIndicator/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 16:03:28,648 - DeepRL - DEBUG - global iter =   40 total obj: 9810.4503
2021-07-12 16:03:35,808 - DeepRL - DEBUG - global iter =   41 total obj: 9799.2919
2021-07-12 16:03:42,890 - DeepRL - DEBUG - global iter =   42 total obj: 9789.2686
2021-07-12 16:03:49,958 - DeepRL - DEBUG - global iter =   43 total obj: 9782.4157
2021-07-12 16:03:57,041 - DeepRL - DEBUG - global iter =   44 total obj: 9774.3152
2021-07-12 16:04:04,113 - DeepRL - DEBUG - global iter =   45 total obj: 9778.4687
2021-07-12 16:04:11,184 - DeepRL - DEBUG - global iter =   46 total obj: 9784.7833
2021-07-12 16:04:18,047 - DeepRL - DEBUG - global iter =   47 total obj: 9823.0176
2021-07-12 16:04:24,247 - DeepRL - DEBUG - global iter =   48 total obj: 9814.5158
2021-07-12 16:04:30,366 - DeepRL - DEBUG - global iter =   49 total obj: 9784.4377
2021-07-12 16:04:30,367 - DeepRL - DEBUG - opt finished.
2021-07-12 16:04:30,368 - DeepRL - DEBUG - started testing...
2021-07-12 16:04:31,927 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/exploreExploit_blockIndicator/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
          0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
          0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
          1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
          0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
          1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
          1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
          1, 0, 0, 1, 1, 0]]),
  'state': array([[[  1,   2],
          [  1,   2],
          [  1,   8],
          [  8,   1],
          [  1,   2],
          [  2,   1],
          [  2,   8],
          [  1,   2],
          [  8,   1],
          [  1,   2],
          [  2,   8],
          [  1,   8],
          [  8,   1],
          [  8,   1],
          [  2,   8],
          [  2,   8],
          [ 15,  22],
          [ 22,  15],
          [ 22,  15],
